In [26]:
class test:
    def __init__(self,*args):
        if args:
            print(args[0])
        pass
        #return self.call()
    def call(self):
        return {'hola':'mundo'}

In [28]:
h=test()

In [21]:
h.call()

{'hola': 'mundo'}

In [120]:
class profile:
    sample={"advisors":[{"ids":[{"value":"INSPIRE-00133260",
                                    "schema":"INSPIRE ID"}],
                "name":"Valle, Jose W.F.",
                "hidden":False,
                "record":{"$ref":"https://inspirehep.net/api/authors/985058"},
                "degree_type":"phd","curated_relation":False},
               {"name":"Ponce, William A.","hidden":False,
                "degree_type":"master",
                "curated_relation":False}],
                "email_addresses":[{"value":"restrepo@udea.edu.co","current":True}],
    "positions":[{"rank":"SENIOR",
                  "hidden":False,
                  "record":{"$ref":"https://inspirehep.net/api/institutions/903906"},
                  "current":True,
                  "start_date":"2004",
                  "institution":"Antioquia U.",
                  "curated_relation":True},
                 {"rank":"PHD",
                  "hidden":False,
                  "record":{"$ref":"https://inspirehep.net/api/institutions/907907"},
                  "current":False,
                  "end_date":"2001","start_date":"1997",
                  "institution":"Valencia U., IFIC","curated_relation":True}],
    "ids":[{"value":"D.Restrepo.1","schema":"INSPIRE BAI"},
           {"value":"INSPIRE-00119748","schema":"INSPIRE ID"},
           {"value":"0000-0001-6455-5564","schema":"ORCID"},
           {"value":"diego-restrepo-209b7927","schema":"LINKEDIN"}],
    "name":{"value":"Restrepo Quintero, Diego Alejandro",
            "preferred_name":"Diego Restrepo"},
    "stub":False,
    "urls":[{"value":"http://gfif.udea.edu.co"},
            {"value":"https://scholar.google.com/citations?user=1sKULCoAAAAJ"},
            {"value":"https://www.researchgate.net/profile/Diego_Restrepo2"}],
    "status":"active",
    "$schema":"https://inspirehep.net/schemas/records/authors.json",
    "deleted":False,
    "control_number":991924,
    "legacy_version":"20210323213044.0",
    "arxiv_categories":["hep-ph"],
    "legacy_creation_date":"1999-08-23"}
    def __init__(self,*args):
        if args:
            if args[0].is_author:
                self.is_author=True
                self.name=args[0].name
        else:
            self.name=''            
            
    def author(self,p):
        '''
        Select:
        * current position
        * non-INSPIRE ids
        * name
        * control_number
        * arxiv_categories
        '''
        self.is_author=True
        self.name=p.get('name')
        return {'name':self.name}
        
        

In [121]:
au=profile()

In [122]:
au.name

''

In [123]:
p=au.sample

In [124]:
au.author(p)

{'name': {'value': 'Restrepo Quintero, Diego Alejandro',
  'preferred_name': 'Diego Restrepo'}}

In [125]:
au.is_author

True

In [126]:
au.name

{'value': 'Restrepo Quintero, Diego Alejandro',
 'preferred_name': 'Diego Restrepo'}

In [127]:
nau=profile(au)

In [128]:
nau.name

{'value': 'Restrepo Quintero, Diego Alejandro',
 'preferred_name': 'Diego Restrepo'}

In [6]:
#in general len(l)!=0
authors=inspire.literature(l)

authors → list of authors object
```
[a_1,a_2,...a_n]
```
where `a_i` is just a dictionary which is either created or updated

In [ ]:
aus=inspire.legacy_instituion(legacy_instituion)
aus=inspire.country(country)
aus=inspire.authors(inspire_authors)
au=inspire.author(inspire_author)

In [ ]:
au =inspire.profile(profile) # → a_i

Input:

In [ ]:
authors=inspire.read_json('authors.json')
authors=inspire.read_list(author_list)

In [4]:
Can be exported to JSON or Excel

{'id': '991924',
 'uuid': 'c233167c-7187-4235-979d-0f5bb68aab97',
 'metadata': {'advisors': [{'ids': [{'value': 'INSPIRE-00133260',
      'schema': 'INSPIRE ID'}],
    'name': 'Valle, Jose W.F.',
    'hidden': False,
    'record': {'$ref': 'https://inspirehep.net/api/authors/985058'},
    'degree_type': 'other',
    'curated_relation': False},
   {'name': 'Ponce, William A.',
    'hidden': False,
    'degree_type': 'master',
    'curated_relation': False}],
  'email_addresses': [{'value': 'restrepo@udea.edu.co', 'current': True}],
  'positions': [{'rank': 'SENIOR',
    'hidden': False,
    'record': {'$ref': 'https://inspirehep.net/api/institutions/903906'},
    'current': True,
    'start_date': '2004',
    'institution': 'Antioquia U.',
    'curated_relation': True},
   {'rank': 'VISITOR',
    'hidden': False,
    'record': {'$ref': 'https://inspirehep.net/api/institutions/902714'},
    'current': False,
    'end_date': '2022',
    'start_date': '2022',
    'institution': 'Campinas S